<a href="https://colab.research.google.com/github/Kavyapm1960/project/blob/main/eng_to_fr_trans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def load_data(filename):
    with open(filename, encoding='utf-8') as file:
        text = file.read()
    return text.split('\n')

def tokenize(sentences):
    tokenizer = Tokenizer(char_level=False)
    tokenizer.fit_on_texts(sentences)
    return tokenizer, tokenizer.texts_to_sequences(sentences)

def pad(sequences, maxlen=None):
    return pad_sequences(sequences, padding='post', maxlen=maxlen)

english_sentences = load_data('/content/drive/MyDrive/small_vocab_en (2).txt')
french_sentences = load_data('/content/drive/MyDrive/small_vocab_fr (1).txt')

english_tokenizer, english_tokenized = tokenize(english_sentences)
french_tokenizer, french_tokenized = tokenize(french_sentences)

max_english_length = max(len(sentence) for sentence in english_tokenized)
max_french_length = max(len(sentence) for sentence in french_tokenized)

english_padded = pad(english_tokenized, maxlen=max_english_length)
french_padded = pad(french_tokenized, maxlen=max_french_length)

encoder_input_data = np.array(english_padded)
decoder_input_data = np.array(french_padded)
decoder_output_data = np.zeros_like(decoder_input_data)
decoder_output_data[:, :-1] = decoder_input_data[:, 1:]

embedding_size = 256
lstm_units = 512
num_encoder_tokens = len(english_tokenizer.word_index) + 1
num_decoder_tokens = len(french_tokenizer.word_index) + 1

encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding_layer = Embedding(num_decoder_tokens, embedding_size)
decoder_embedding = decoder_embedding_layer(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

decoder_output_one_hot = np.zeros(
    (len(french_sentences), max_french_length, num_decoder_tokens),
    dtype='float32'
)

for i, sentence in enumerate(decoder_output_data):
    for t, word_idx in enumerate(sentence):
        if word_idx > 0:
            decoder_output_one_hot[i, t, word_idx] = 1

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_output_one_hot,
    batch_size=512,
    epochs=50,
    validation_split=0.2
)

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_embedding = decoder_embedding_layer(decoder_inputs)

decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    # Start the target sequence with the first word index or another suitable starting point
    target_seq = np.zeros((1, 1))

    # You might want to start with the most common word or another heuristic
    # For simplicity, we start with the first word index; adjust as needed
    start_token_index = 1  # or another index that makes sense for your dataset
    target_seq[0, 0] = start_token_index

    decoded_sentence = ''
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_french_tokenizer.get(sampled_token_index, '')
        if sampled_word == '<end>' or len(decoded_sentence) > max_french_length:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word
            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = sampled_token_index
            states_value = [h, c]
    return decoded_sentence.strip()

reverse_french_tokenizer = {i: word for word, i in french_tokenizer.word_index.items()}

while True:
    input_sentence = input("Enter English sentence to translate (or type 'quit' to exit): ")
    if input_sentence.lower() == 'quit':
        break
    input_seq = english_tokenizer.texts_to_sequences([input_sentence])
    padded_input_seq = pad(input_seq, maxlen=max_english_length)
    translation = decode_sequence(padded_input_seq)
    print("French translation:", translation)

Epoch 1/50
216/216 [==============================] - 26s 99ms/step - loss: 2.1374 - accuracy: 0.0941 - val_loss: 1.8429 - val_accuracy: 0.1145
Epoch 2/50
216/216 [==============================] - 15s 71ms/step - loss: 1.7188 - accuracy: 0.1553 - val_loss: 1.7359 - val_accuracy: 0.1672
Epoch 3/50
216/216 [==============================] - 15s 71ms/step - loss: 1.5219 - accuracy: 0.1989 - val_loss: 1.4234 - val_accuracy: 0.2121
Epoch 4/50
216/216 [==============================] - 16s 72ms/step - loss: 1.3620 - accuracy: 0.2346 - val_loss: 1.2850 - val_accuracy: 0.2576
Epoch 5/50
216/216 [==============================] - 16s 74ms/step - loss: 1.2316 - accuracy: 0.2655 - val_loss: 1.1961 - val_accuracy: 0.2742
Epoch 6/50
216/216 [==============================] - 16s 73ms/step - loss: 1.1280 - accuracy: 0.2861 - val_loss: 1.0763 - val_accuracy: 0.2905
Epoch 7/50
216/216 [==============================] - 16s 72ms/step - loss: 1.0547 - accuracy: 0.2987 - val_loss: 1.0076 - val_accuracy: